In this notebook, we explore basic analysis of the positions and speeds to detect similarities and differences between behaviouirs.

We start importing some libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

from navground import core, sim

plots_dir = os.path.join("plots", "metrics")
experiment_dir = "experiments"
os.makedirs(plots_dir, exist_ok=True)
os.makedirs("experiments", exist_ok=True)

In [ ]:
num_steps = 400
side=2.5
num_agents = 31 # 38
num_runs = 13
behaviour_list = ["ORCA", "HL", "HRVO"]
color_behaviour = {}
for i, behaviour in enumerate(behaviour_list):
    color_behaviour[behaviour] = mpl.colormaps["Set1"](i / (len(behaviour_list) +1))
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"measurements_{behaviour}.h5")
    yaml = f"""
    steps: {num_steps}
    time_step: 0.1
    record_pose: true
    record_twist: true
    runs: {num_runs}
    record_collisions: true
    record_deadlocks: true
    record_safety_violation: true
    record_efficacy: true
    scenario:
      type: Cross 
      side: {side} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.08
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.166
          behavior:
            type: {behaviour}
            optimal_speed: 
                sampler: normal
                mean: 0.4
                std_dev: 0.05
            horizon: 5.0
            safety_margin: 
                sampler: normal
                mean: 0.2
                std_dev: 0.05
          state_estimation:
            type: Bounded
            range: 5.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run(keep=False, data_path=path)
    del experiment

In [ ]:
runs = {}
# Reload simulations
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"measurements_{behaviour}.h5")
    recorded_experiment = sim.RecordedExperiment(path)
    runs[behaviour] = recorded_experiment.runs

Visualise a simulation

In [ ]:
from navground.sim.ui import WebUI
from navground.sim.notebook import notebook_view
from navground.sim.replay import RealTimeReplay

web_ui = WebUI(host='127.0.0.1', max_rate=-1)
await web_ui.prepare()

In [ ]:
steps_list = list(range(100, 220, 5))
variables_behaviour = {}
for j, behaviour in enumerate(behaviour_list):
    variables_list = []
    for ridx in range(num_runs):
        variables_run = []
        ps = np.array(runs[behaviour][ridx].poses)
        twists = np.array(runs[behaviour][ridx].twists)
        for idx, step in enumerate(steps_list):
            X = ps[step]
            vel_X = twists[step]
            variables_run.append(np.hstack((np.mean(X, axis=0), np.std(X, axis=0), np.mean(vel_X, axis=0), np.std(vel_X, axis=0))))
        # end for 
        variables_list.append(np.array(variables_run).transpose())
    # end for
    variables_behaviour[behaviour] = np.array(variables_list)

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=6, figsize=(12,8))
labels = np.array([
    ["x mean", "y mean", "angle mean", "x std", "y std", "angule std"],
    ["v_x mean", "v_y mean", "angle speed mean", "v_x std", "v_y std", "angular speed std"]
]).transpose()
for irow in range(labels.shape[0]):
    for icol in range(labels.shape[1]):
        for behaviour in behaviour_list:
            for variables_run in variables_behaviour[behaviour]:
                ax[irow, icol].set_title(labels[irow, icol])
                ax[irow, icol].plot(list(range(variables_run.shape[1])), variables_run[icol*6 + irow], color=color_behaviour[behaviour], label=behaviour)

plt.tight_layout()
# Remove duplicate labels 
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.savefig(os.sep.join((plots_dir, "basic_metrics_signals.png")))

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(12,4))
labels = np.array([
    ["x mean", "y mean", "angle mean", "x std", "y std", "angule std"],
    ["v_x mean", "v_y mean", "angle speed mean", "v_x std", "v_y std", "angular speed std"]
])
variables = np.vstack([variables_behaviour[behaviour] for behaviour in behaviour_list])
for irow in range(labels.shape[0]):
    for icol in range(labels.shape[1]):
        ax[irow, icol].set_title(labels[irow, icol])
        pca = PCA(n_components=2)
        Y = pca.fit_transform(variables[:, irow*6 + icol, :])
        Y_dict = {}
        for i, behaviour in enumerate(behaviour_list):
            Y_dict[behaviour] = Y[num_runs*i:num_runs*(i+1)]
        # end for
        for behaviour in behaviour_list:
            ax[irow, icol].scatter(Y_dict[behaviour][:,0], Y_dict[behaviour][:,1], color=color_behaviour[behaviour], label=behaviour)

# Remove duplicate labels 
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behaviour_list))
plt.tight_layout()
plt.savefig(os.sep.join((plots_dir, "basic_metrics_PCA.png")))

Let us see how these simulations relate to efficacy and number of collisions.

In [ ]:
import pandas as pd

def count_deadlocks(deadlock_time, initial_time, final_time):
    is_deadlocked = np.logical_and(deadlock_time > initial_time, deadlock_time < (final_time - 5.0))
    return sum(is_deadlocked)

def extract_data(experiment, initial_step, final_step):
    collisions = []
    deadlocks = []
    efficacy = []
    sms = []
    seeds = []
    for i, run in experiment.runs.items():
        world = run.world
        initial_time, final_time = initial_step*run.time_step, final_step*run.time_step
        deadlocks.append(count_deadlocks(np.array(run.deadlocks), initial_time, final_time))
        collisions.append(np.sum(np.logical_and(
            initial_step < run.collisions[:,0], run.collisions[:,0] < final_step
        )))
        efficacy.append(np.array(run.efficacy[initial_step:final_step]).mean())

    df = pd.DataFrame({
        'deadlocks': deadlocks,
        'collisions': collisions,
        'efficacy': efficacy})
    df['safe'] = (df.collisions == 0).astype(int)
    df['fluid'] = (df.deadlocks == 0).astype(int)
    df['ok'] = ((df.deadlocks == 0) & (df.collisions == 0)).astype(int)
    return df

In [ ]:
initial_step = steps_list[0]
final_step = steps_list[-1]
deadlocks = {}
collisions = {}
efficacy = {}
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"measurements_{behaviour}.h5")
    recorded_experiment = sim.RecordedExperiment(path)
    df = extract_data(recorded_experiment, initial_step, final_step)
    collisions[behaviour] =  list(df.collisions)
    deadlocks[behaviour] = list(df.deadlocks)
    efficacy[behaviour] = list(df.efficacy)

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), ncols=3)
for behaviour in behaviour_list:
    ax[0].scatter(deadlocks[behaviour], collisions[behaviour], color=color_behaviour[behaviour], label=behaviour)
    ax[0].set_xlabel("deadlocks")
    ax[0].set_ylabel("collisions")
    ax[1].scatter(efficacy[behaviour], collisions[behaviour], color=color_behaviour[behaviour], label=behaviour)
    ax[1].set_xlabel("efficacy")
    ax[1].set_ylabel("collisions")
    ax[2].scatter(efficacy[behaviour], deadlocks[behaviour], color=color_behaviour[behaviour], label=behaviour)
    ax[2].set_xlabel("efficacy")
    ax[2].set_ylabel("deadlocks")
# end for
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behaviour_list))
plt.tight_layout()
plt.savefig(os.sep.join((plots_dir, "efficiency_collisions_cross.png")))

In [ ]:
deadlocks["HL"]